In [233]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score

Random Sparse Projections

In [235]:
def generate_sparse_projection(input_dim, hidden_dim, connections_per_hidden):
    """
    Generates a sparse binary projection matrix.
    
    Args:
        input_dim (int): Number of input features.
        hidden_dim (int): Number of hidden neurons (KC nodes).
        connections_per_hidden (int): Non-zero connections per hidden node.
    
    Returns:
        torch.Tensor: Sparse binary weight matrix of shape (hidden_dim, input_dim).
    """
    projection = np.zeros((hidden_dim, input_dim))
    for i in range(hidden_dim):
        selected_indices = np.random.choice(input_dim, connections_per_hidden, replace=False)
        projection[i, selected_indices] = 1
    return torch.tensor(projection, dtype=torch.float32)

Global Inhibition Function

In [237]:
def global_inhibition(hidden_activations):
    """
    Applies global inhibition to enforce sparsity in activations.
    
    Args:
        hidden_activations (torch.Tensor): Hidden layer activations.
    
    Returns:
        torch.Tensor: Inhibited activations.
    """
    inhibtion_factor=1 
    mean_activation = hidden_activations.mean(dim=1, keepdim=True)
    inhibited_activations = hidden_activations - inhibtion_factor*mean_activation
    return inhibited_activations.clamp(min=0)  # ReLU equivalent

Forward Pass

In [239]:
def forward_pass(x, projection_matrix, output_weights):
    """
    Performs a forward pass through the KCNet.
    
    Args:
        x (torch.Tensor): Input data of shape (batch_size, input_dim).
        projection_matrix (torch.Tensor): Sparse binary projection matrix.
        output_weights (torch.Tensor): Weights from hidden layer to output layer.
    
    Returns:
        torch.Tensor: Predicted output of shape (batch_size, num_classes).
    """
    # Step 1: Random projection
    hidden_pre_activation = x @ projection_matrix.T
    
    # Step 2: Global inhibition
    hidden_activations = global_inhibition(hidden_pre_activation)
    
    # Step 3: Linear output layer
    output = hidden_activations @ output_weights
    return output


Ridge Regression for Output Weights

In [241]:
def ridge_regression(hidden_layer_outputs, targets, lambda_reg=0.01):
    """
    Solves for output weights using ridge regression.
    
    Args:
        hidden_layer_outputs (torch.Tensor): Hidden layer activations (H).
        targets (torch.Tensor): Target outputs (Y).
        lambda_reg (float): Regularization term.
    
    Returns:
        torch.Tensor: Optimized output weights (Beta).
    """
    H_T = hidden_layer_outputs.T  # Shape (2000, 100)
    regularization = lambda_reg * torch.eye(H_T.shape[0])  # Matches hidden nodes (2000, 2000)
    # Ridge regression
    beta = torch.inverse(H_T @ hidden_layer_outputs + regularization) @ H_T @ targets
    return beta

KCNET Class For Summarizing Whole Functionals

In [243]:
class KCNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, connections_per_hidden):
        """
        Initializes the KCNet.
        
        Args:
            input_dim (int): Number of input features.
            hidden_dim (int): Number of hidden neurons (KC nodes).
            output_dim (int): Number of output classes.
            connections_per_hidden (int): Non-zero connections per hidden node.
        """
        super(KCNet, self).__init__()
        self.projection_matrix = generate_sparse_projection(input_dim, hidden_dim, connections_per_hidden)
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.output_weights = None
    
    def forward(self, x):
        """
        Forward pass through the KCNet.
        
        Args:
            x (torch.Tensor): Input data.
        
        Returns:
            torch.Tensor: Predicted output.
        """
        return forward_pass(x, self.projection_matrix, self.output_weights)
    
    def train_output_layer(self, hidden_layer_outputs, targets, lambda_reg=0.01):
        """
        Trains the output layer weights using ridge regression.
        
        Args:
            hidden_layer_outputs (torch.Tensor): Hidden layer activations (H).
            targets (torch.Tensor): Target outputs (Y).
            lambda_reg (float): Regularization term.
        """
        self.output_weights = ridge_regression(hidden_layer_outputs, targets, lambda_reg)



Testing on Random Dataset

In [245]:
# Parameters
input_dim = 50      # Example input features
hidden_dim = 2000   # Kenyon cell count
output_dim = 10     # Number of output classes
connections_per_hidden = 7  # Sparse connections
lambda_reg = 0.1    # Regularization strength

n_runs = 10
results = []
for run in range(n_runs):
    # Initialize KCNet
    kcnet = KCNet(input_dim, hidden_dim, output_dim, connections_per_hidden)
    
    # Example data (random for illustration)
    X = torch.rand(100, input_dim)  # 100 samples, 50 features
    y = torch.randint(0, output_dim, (100,))  # Random targets
    y_one_hot = torch.nn.functional.one_hot(y, num_classes=output_dim).float()
    
    # Step 1: Hidden layer activations
    hidden_layer_outputs = torch.relu(X @ kcnet.projection_matrix.T)
    
    # Step 2: Train output layer
    kcnet.train_output_layer(hidden_layer_outputs, y_one_hot, lambda_reg)
    
    # Step 3: Predictions
    predictions = kcnet(X).argmax(dim=1)
    
    # Step 4: Evaluation
    accuracy = accuracy_score(y.numpy(), predictions.numpy())
    print(f"Accuracy: {accuracy * 100:.2f}%")
    results.append(accuracy)

# Calculate average and standard deviation
average_result = np.mean(results)
std_deviation = np.std(results)

# Print results
print(f"Average Accuracy: {average_result:.4f} ± {std_deviation:.4f}")


Accuracy: 91.00%
Accuracy: 87.00%
Accuracy: 92.00%
Accuracy: 96.00%
Accuracy: 89.00%
Accuracy: 87.00%
Accuracy: 91.00%
Accuracy: 90.00%
Accuracy: 86.00%
Accuracy: 91.00%
Average Accuracy: 0.9000 ± 0.0279
